In [ ]:
# 基础环境检查与库安装
import sys
import os

# 定义需要安装的库
required_packages = ["pymupdf", "pytesseract", "spacy", "pandas", "openpyxl", "requests", "pillow"]

# 自动安装缺失库 (让代码更稳健，换了电脑也能跑)
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"正在安装缺失库: {package}...")
        !{sys.executable} -m pip install {package} -q

# 下载 spaCy 模型
try:
    import spacy
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("正在下载 spaCy 英文模型...")
    !{sys.executable} -m spacy download en_core_web_sm
    nlp = spacy.load("en_core_web_sm")

print("✅ 所有环境配置检查完毕！")

In [ ]:
# Cell 2: 核心功能库导入 (修复版 - 增强下载功能)
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
import pandas as pd
import requests
import re
import os
from collections import Counter

# 【重要配置】Windows 用户若报错，请取消下面注释并修改路径
pytesseract.pytesseract.tesseract_cmd = r'd:\Tesseract-OCR\tesseract.exe'

def download_sample_pdf(url, filename="sample_paper.pdf"):
    """
    稳健的 PDF 下载函数：
    1. 包含浏览器伪装 (User-Agent)。
    2. 检查文件是否已存在。
    3. 检查服务器返回的是否真的是 PDF (Content-Type)。
    """
    if os.path.exists(filename):
        # 检查文件大小，如果文件太小（小于 10KB），可能是之前下载坏的，强制重新下载
        file_size = os.path.getsize(filename)
        if file_size > 10240: 
            print(f"📂 本地文件已存在且有效 ({file_size/1024:.1f} KB): {filename}")
            return filename
        else:
            print(f"⚠️ 发现损坏的旧文件 ({file_size} bytes)，正在删除并重新下载...")
            os.remove(filename)
    
    print(f"⬇️ 正在下载测试文献: {url}...")
    
    # === 关键修复 1: 添加浏览器伪装头 (User-Agent) ===
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Referer": "https://www.google.com/"
    }

    try:
        # 使用 headers 发起请求
        response = requests.get(url, headers=headers, stream=True, timeout=15)
        response.raise_for_status()
        
        # === 关键修复 2: 检查内容类型 (Content-Type) ===
        content_type = response.headers.get("Content-Type", "").lower()
        if "pdf" not in content_type and "application/octet-stream" not in content_type:
            print(f"❌ 下载失败：服务器返回的不是 PDF，而是 {content_type}")
            print("可能是反爬虫拦截，建议手动下载 PDF 并拖入文件夹。")
            return None

        # 写入文件
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        print(f"✅ 下载完成！文件已保存: {filename}")
        return filename

    except Exception as e:
        print(f"❌ 下载过程出错: {e}")
        return None

def smart_pdf_parser(pdf_path):
    """能够同时处理文本版和扫描版 PDF 的解析器"""
    if not pdf_path or not os.path.exists(pdf_path): 
        print("❌ 错误：找不到 PDF 文件路径。")
        return ""
    
    try:
        doc = fitz.open(pdf_path)
    except Exception as e:
        # 这里能捕捉到“文件损坏”的错误
        print(f"❌ 无法打开 PDF 文件 (文件可能已损坏): {e}")
        print("💡 建议：删除目录下的 .pdf 文件，重新运行下载代码，或手动上传一个正常的 PDF。")
        return ""

    full_text = []
    print(f"🚀 开始处理文件: {pdf_path} (共 {len(doc)} 页)...")

    for i, page in enumerate(doc):
        # 1. 尝试直接提取
        text = page.get_text()
        
        # 2. 扫描件检测逻辑
        if len(text.strip()) < 50:
            print(f"  ⚠️ 第 {i+1} 页疑似扫描/图表，启用 OCR 识别...")
            try:
                pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))
                img = Image.open(io.BytesIO(pix.tobytes("png")))
                text = pytesseract.image_to_string(img, lang='eng')
            except Exception as e:
                print(f"  ❌ OCR 失败 (第 {i+1} 页): {e}")
                # 如果 OCR 失败，保留空文本，防止程序崩溃
                text = "" 
        
        full_text.append(text)
    
    return "\n".join(full_text)

# === 运行准备 ===
# 使用 Attention Is All You Need 的备用链接 (Semantic Scholar 通常比 arXiv 稍微宽松一点，或者坚持用 arXiv 但带 Header)
# 这里的 URL 依然用 arXiv，但配合了我们上面的 headers 修复
pdf_url = "https://arxiv.org/abs/2601.09348"

# 运行下载
local_pdf = download_sample_pdf(pdf_url, "transformer_paper.pdf")

# 如果自动下载失败，使用备用方案提示
if local_pdf:
    # 执行解析
    raw_content = smart_pdf_parser(local_pdf)
    if raw_content:
        print(f"✅ 解析完成，共提取字符数: {len(raw_content)}")
    else:
        print("❌ 解析中断：未能提取到内容。")
else:
    print("❌ 无法进行解析，因为 PDF 下载失败。")





    





In [ ]:
# 文本清洗与术语挖掘 (Text Mining)
def extract_terminology(text, top_k=50):
    if not text: return pd.DataFrame()
    
    print("🧹 正在清洗文本并进行 NLP 分析...")
    
    # 清洗：修复断词 (ex- ample -> example)
    text = re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)
    # 清洗：合并换行符
    clean_text = text.replace('\n', ' ')
    
    # NLP 处理 (截取前 50万字符防止内存溢出)
    doc = nlp(clean_text[:500000])
    
    candidates = []
    sentences = list(doc.sents)
    
    # 提取名词短语
    for chunk in doc.noun_chunks:
        term = chunk.text.lower().strip()
        # 筛选规则：长度>2, 含空格, 无数字, 非代词开头
        if len(term) > 2 and " " in term and not any(c.isdigit() for c in term):
             if chunk[0].pos_ != "PRON":
                candidates.append(term)
    
    # 统计频率
    counts = Counter(candidates)
    
    # 构建数据表 (含例句)
    data = []
    print(f"🔍 正在为前 {top_k} 个术语匹配例句...")
    
    for term, freq in counts.most_common(top_k):
        # 查找例句
        context = "Context not found"
        for sent in sentences:
            if term in sent.text.lower():
                context = sent.text.strip().replace('\n', ' ')
                break
        
        data.append({
            "Source Term": term,
            "Frequency": freq,
            "Context (Example)": context,
            "Target Term (CN)": "" # 预留给人工填空
        })
        
    return pd.DataFrame(data)

# 执行提取
df_result = extract_terminology(raw_content)
display(df_result.head())


In [ ]:
# 结果导出
output_file = "Term_Glossary.xlsx"
try:
    df_result.to_excel(output_file, index=False)
    print(f"🎉 成功！文件已保存至: {os.path.abspath(output_file)}")
    print("💡 下一步：在左侧文件栏右键点击下载，或直接使用 Excel 打开。")
except Exception as e:
    print(f"❌ 保存失败: {e}")